In [1]:
from platform import python_version
from pulp import *
import pulp

print("Python version: ", python_version())
print("PuLP version: ", pulp.__version__)

Python version:  3.9.12
PuLP version:  2.7.0


In [7]:
def mps2PuLP(filename):
    # read mps file
    mpsFile = open(filename, "r")
    mpsLine = mpsFile.readlines()
    mpsLine = [s.replace('\n', '') for s in mpsLine]

    readRow, readCol, readRHS, readBnd, readObj = False, False, False, False, False
    equations, variables, objEq = {}, set(), None
    LHSeq, RHSeq, upBound, lowBound = {}, {}, {}, {}

    for i in range(len(mpsLine)):
        string = mpsLine[i]  
        if  string == 'ROWS':
            readRow = True
        elif string == 'COLUMNS':
            readRow = False
            readCol = True
            LHS = {k: [] for k in equations.keys()}
            RHS = {k: [] for k in equations.keys()}
        elif string == 'RHS':
            readCol = False
            readRHS = True
        elif string == 'BOUNDS':
            readRHS = False
            readBnd = True

        if readRow and string != 'ROWS':
            relation, eq = string.split()
            equations[eq] = relation
            if not readObj and relation == 'N': ObjEq = eq

        if readCol and string != 'COLUMNS':
            strList = string.split()
            var = strList[0]
            variables.add(var)
            for i in range(0, len(strList)//2):
                LHS[strList[i*2+1]].append((var, strList[i*2+2]))

        if readRHS and string != 'RHS':
            strList = string.split()
            for i in range(0, len(strList)//2):
                RHS[strList[i*2+1]] = strList[i*2+2]

        if readBnd and string != 'BOUNDS':
            strList = string.split()
            if strList[0] == 'UP':
                upBound[strList[2]] = strList[3]
            elif strList[0] == 'LO':
                lowBound[strList[2]] = strList[3]

    mpsFile.close()
    
    ### convert into PuLP ###
    model = LpProblem("mps_Model", LpMinimize)

    # variables
    x = LpVariable.dicts("var_", variables, cat='continuous')

    # constraints
    for eq in equations:
        currLHS = lpSum(x[v]*float(coef) for v, coef in LHS[eq])

        if eq == ObjEq:
            model += currLHS

        if equations[eq] == 'E':
            model += currLHS == float(RHS[eq])
        elif equations[eq] == 'G':
            model += currLHS >= float(RHS[eq])
        elif equations[eq] == 'L':
            model += currLHS <= float(RHS[eq])

    # variables: constraints
    for k in upBound.keys():
        x[k].upBound = float(upBound[k])
    for k in lowBound.keys():
        x[k].lowBound = float(lowBound[k])
    
    return model, x

In [16]:
model1, x = mps2PuLP("testprob.mps")

In [9]:
model1.solve()

1

In [10]:
value(model1.objective)

54.0

In [14]:
for k in x.keys():
    print(k+": "+str(x[k].varValue))

XONE: 4.0
YTWO: -1.0
ZTHREE: 6.0
